In [1]:
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn_extra.cluster import KMedoids
import geopandas as gpd 
import numpy as np
import pandas as pd
from pyecharts.globals import CurrentConfig

CurrentConfig.ONLINE_HOST="https://cdn.kesci.com/lib/pyecharts_assets/"
%matplotlib inline

sf = gpd.read_file('./data_for_paper_final_version/Grids with hash rate fluc index(201806-201905)/Grids with hash rate fluc index(201806-201905).shp')
fluc = sf[sf['HrateAvg']>=1500]
code_hr = []
for index, row in fluc.iterrows():
    colums = []
    for colum in fluc.columns[3:-2]:
#         colum = str(month)+'hsfluc'
        code = 0
        if row[colum] < -0.75:
            code = 1
        elif row[colum] < -0.25:
            code = 2
        elif row[colum] <= 0.25:
            code = 3
        elif row[colum] <= 0.75:
            code = 4
        elif row[colum] <= 1:
            code = 5
        else:
            code = 6
        colums.append(code)
    code_hr.append(colums)

In [2]:
y_pred = KMedoids(n_clusters=12,method='pam', init='build').fit(code_hr)

In [ ]:
'''
source code for sklearn_extra.cluster.KMedoids
https://scikit-learn-extra.readthedocs.io/en/stable/_modules/sklearn_extra/cluster/_k_medoids.html#KMedoids

class KMedoids(BaseEstimator, ClusterMixin, TransformerMixin):
    
    def __init__(
        self,
        n_clusters=8,
        metric="euclidean",
        method="alternate",
        init="heuristic",
        max_iter=300,
        random_state=None,
    ):
        self.n_clusters = n_clusters
        self.metric = metric
        self.method = method
        self.init = init
        self.max_iter = max_iter
        self.random_state = random_state

    def _check_nonnegative_int(self, value, desc, strict=True):
        """Validates if value is a valid integer > 0"""
        if strict:
            negative = (value is None) or (value <= 0)
        else:
            negative = (value is None) or (value < 0)
        if negative or not isinstance(value, (int, np.integer)):
            raise ValueError(
                "%s should be a nonnegative integer. "
                "%s was given" % (desc, value)
            )

    def _check_init_args(self):
        """Validates the input arguments. """

        # Check n_clusters and max_iter
        self._check_nonnegative_int(self.n_clusters, "n_clusters")
        self._check_nonnegative_int(self.max_iter, "max_iter", False)

        # Check init
        init_methods = ["random", "heuristic", "k-medoids++", "build"]
        if self.init not in init_methods:
            raise ValueError(
                "init needs to be one of "
                + "the following: "
                + "%s" % init_methods
            )

    def fit(self, X, y=None):
       
        random_state_ = check_random_state(self.random_state)

        self._check_init_args()
        X = check_array(X, accept_sparse=["csr", "csc"])
        if self.n_clusters > X.shape[0]:
            raise ValueError(
                "The number of medoids (%d) must be less "
                "than the number of samples %d."
                % (self.n_clusters, X.shape[0])
            )

        D = pairwise_distances(X, metric=self.metric)
        medoid_idxs = self._initialize_medoids(
            D, self.n_clusters, random_state_
        )
        labels = None

        if self.method == "pam":
            # Compute the distance to the first and second closest points
            # among medoids.
            Djs, Ejs = np.sort(D[medoid_idxs], axis=0)[[0, 1]]

        
        for self.n_iter_ in range(0, self.max_iter):
            old_medoid_idxs = np.copy(medoid_idxs)
            labels = np.argmin(D[medoid_idxs, :], axis=0)

            if self.method == "alternate":
                # Update medoids with the new cluster indices
                self._update_medoid_idxs_in_place(D, labels, medoid_idxs)
            elif self.method == "pam":
                not_medoid_idxs = np.delete(np.arange(len(D)), medoid_idxs)
                optimal_swap = _compute_optimal_swap(
                    D,
                    medoid_idxs.astype(np.intc),
                    not_medoid_idxs.astype(np.intc),
                    Djs,
                    Ejs,
                    self.n_clusters,
                )
                if optimal_swap is not None:
                    i, j, _ = optimal_swap
                    medoid_idxs[medoid_idxs == i] = j

                    # update Djs and Ejs with new medoids
                    Djs, Ejs = np.sort(D[medoid_idxs], axis=0)[[0, 1]]
            else:
                raise ValueError(
                    f"method={self.method} is not supported. Supported methods "
                    f"are 'pam' and 'alternate'."
                )

            if np.all(old_medoid_idxs == medoid_idxs):
                break
            elif self.n_iter_ == self.max_iter - 1:
                warnings.warn(
                    "Maximum number of iteration reached before "
                    "convergence. Consider increasing max_iter to "
                    "improve the fit.",
                    ConvergenceWarning,
                )

        # Set the resulting instance variables.
        if self.metric == "precomputed":
            self.cluster_centers_ = None
        else:
            self.cluster_centers_ = X[medoid_idxs]

        # Expose labels_ which are the assignments of
        # the training data to clusters
        self.labels_ = np.argmin(D[medoid_idxs, :], axis=0)
        self.medoid_indices_ = medoid_idxs
        self.inertia_ = self._compute_inertia(self.transform(X))

        # Return self to enable method chaining
        return self

    def _update_medoid_idxs_in_place(self, D, labels, medoid_idxs):
        """In-place update of the medoid indices"""

        # Update the medoids for each cluster
        for k in range(self.n_clusters):
            # Extract the distance matrix between the data points
            # inside the cluster k
            cluster_k_idxs = np.where(labels == k)[0]

            if len(cluster_k_idxs) == 0:
                warnings.warn(
                    "Cluster {k} is empty! "
                    "self.labels_[self.medoid_indices_[{k}]] "
                    "may not be labeled with "
                    "its corresponding cluster ({k}).".format(k=k)
                )
                continue

            in_cluster_distances = D[
                cluster_k_idxs, cluster_k_idxs[:, np.newaxis]
            ]

            # Calculate all costs from each point to all others in the cluster
            in_cluster_all_costs = np.sum(in_cluster_distances, axis=1)

            min_cost_idx = np.argmin(in_cluster_all_costs)
            min_cost = in_cluster_all_costs[min_cost_idx]
            curr_cost = in_cluster_all_costs[
                np.argmax(cluster_k_idxs == medoid_idxs[k])
            ]

            # Adopt a new medoid if its distance is smaller then the current
            if min_cost < curr_cost:
                medoid_idxs[k] = cluster_k_idxs[min_cost_idx]

    def _compute_cost(self, D, medoid_idxs):
        """ Compute the cose for a given configuration of the medoids"""
        return self._compute_inertia(D[:, medoid_idxs])

    def transform(self, X):

        X = check_array(X, accept_sparse=["csr", "csc"])

        if self.metric == "precomputed":
            check_is_fitted(self, "medoid_indices_")
            return X[:, self.medoid_indices_]
        else:
            check_is_fitted(self, "cluster_centers_")

            Y = self.cluster_centers_
            return pairwise_distances(X, Y=Y, metric=self.metric)

    def predict(self, X):
       
        X = check_array(X, accept_sparse=["csr", "csc"])

        if self.metric == "precomputed":
            check_is_fitted(self, "medoid_indices_")
            return np.argmin(X[:, self.medoid_indices_], axis=1)
        else:
            check_is_fitted(self, "cluster_centers_")

            # Return data points to clusters based on which cluster assignment
            # yields the smallest distance
            return pairwise_distances_argmin(
                X, Y=self.cluster_centers_, metric=self.metric
            )

    def _compute_inertia(self, distances):
       
        # Define inertia as the sum of the sample-distances
        # to closest cluster centers
        inertia = np.sum(np.min(distances, axis=1))

        return inertia

    def _initialize_medoids(self, D, n_clusters, random_state_):
        """Select initial mediods when beginning clustering."""

        if self.init == "random":  # Random initialization
            # Pick random k medoids as the initial ones.
            medoids = random_state_.choice(len(D), n_clusters)
        elif self.init == "k-medoids++":
            medoids = self._kpp_init(D, n_clusters, random_state_)
        elif self.init == "heuristic":  # Initialization by heuristic
            # Pick K first data points that have the smallest sum distance
            # to every other point. These are the initial medoids.
            medoids = np.argpartition(np.sum(D, axis=1), n_clusters - 1)[
                :n_clusters
            ]
        elif self.init == "build":  # Build initialization
            medoids = _build(D, n_clusters).astype(np.int64)
        else:
            raise ValueError(f"init value '{self.init}' not recognized")

        return medoids

    # Copied from sklearn.cluster.k_means_._k_init
    def _kpp_init(self, D, n_clusters, random_state_, n_local_trials=None):

        n_samples, _ = D.shape

        centers = np.empty(n_clusters, dtype=int)

        # Set the number of local seeding trials if none is given
        if n_local_trials is None:
            # This is what Arthur/Vassilvitskii tried, but did not report
            # specific results for other than mentioning in the conclusion
            # that it helped.
            n_local_trials = 2 + int(np.log(n_clusters))

        center_id = random_state_.randint(n_samples)
        centers[0] = center_id

        # Initialize list of closest distances and calculate current potential
        closest_dist_sq = D[centers[0], :] ** 2
        current_pot = closest_dist_sq.sum()

        # pick the remaining n_clusters-1 points
        for cluster_index in range(1, n_clusters):
            rand_vals = (
                random_state_.random_sample(n_local_trials) * current_pot
            )
            candidate_ids = np.searchsorted(
                stable_cumsum(closest_dist_sq), rand_vals
            )

            # Compute distances to center candidates
            distance_to_candidates = D[candidate_ids, :] ** 2

            # Decide which candidate is the best
            best_candidate = None
            best_pot = None
            best_dist_sq = None
            for trial in range(n_local_trials):
                # Compute potential when including center candidate
                new_dist_sq = np.minimum(
                    closest_dist_sq, distance_to_candidates[trial]
                )
                new_pot = new_dist_sq.sum()

                # Store result if it is the best local trial so far
                if (best_candidate is None) or (new_pot < best_pot):
                    best_candidate = candidate_ids[trial]
                    best_pot = new_pot
                    best_dist_sq = new_dist_sq

            centers[cluster_index] = best_candidate
            current_pot = best_pot
            closest_dist_sq = best_dist_sq

        return centers

'''

In [3]:
y_pred.medoid_indices_

array([ 35, 128,  60, 114,  95, 131, 213, 215, 181, 160,  24,  72],
      dtype=int64)

In [4]:
fluc['cluster'] = y_pred.labels_

In [5]:
fluc.to_file('./data_for_paper_final_version/Grids with hash rate fluc index(201806-201905)/Grids with hash rate fluc index(201806-201905).shp')

In [6]:
import pyecharts.options as opts
from pyecharts.charts import Line

months = [1806,1807,1808, 1809, 1810, 1811, 1812, 1901, 1902, 1903, 1904, 1905] 
x_axis = [str(m) for m in months]

In [7]:
cluster_num = 0
id_num = -1
line=Line()
line.add_xaxis(x_axis)
for i in y_pred.labels_:
    id_num += 1
    if i == cluster_num:
        y_axis1 = fluc[fluc.columns[3:-2]].iloc[id_num]
        line.add_yaxis(str(fluc.iloc[id_num]['Grid_ID']),y_axis1,is_smooth=True,linestyle_opts=opts.LineStyleOpts(width=2),
                       label_opts = opts.LabelOpts(is_show=False))
        line.set_global_opts(yaxis_opts=opts.AxisOpts(name='Fluctuation index', name_location='middle',name_gap=30,min_=-1, max_=11, interval = 1,
                                                     name_textstyle_opts=opts.TextStyleOpts(font_family= 'Times New Roman',font_size=16,color='black',),),
                             legend_opts=opts.LegendOpts(pos_left=30, padding = 90, pos_top=-30, orient='horizontal'))
line.render_notebook()  

In [8]:
cluster_num = 1
id_num = -1
line=Line()
line.add_xaxis(x_axis)
for i in y_pred.labels_:
    id_num += 1
    if i == cluster_num:
        y_axis1 = fluc[fluc.columns[3:-2]].iloc[id_num]
        line.add_yaxis(str(fluc.iloc[id_num]['Grid_ID']),y_axis1,is_smooth=True,linestyle_opts=opts.LineStyleOpts(width=2),
                       label_opts = opts.LabelOpts(is_show=False))
        line.set_global_opts(yaxis_opts=opts.AxisOpts(name='Fluctuation index', name_location='middle',name_gap=30,min_=-1, max_=11, interval = 1,
                                                     name_textstyle_opts=opts.TextStyleOpts(font_family= 'Times New Roman',font_size=16,color='black',),),
                             legend_opts=opts.LegendOpts(pos_left=30, padding = 90, pos_top=-30, orient='horizontal'))
line.render_notebook()  

In [9]:
cluster_num = 2
id_num = -1
line=Line()
line.add_xaxis(x_axis)
for i in y_pred.labels_:
    id_num += 1
    if i == cluster_num:
        y_axis1 = fluc[fluc.columns[3:-2]].iloc[id_num]
        line.add_yaxis(str(fluc.iloc[id_num]['Grid_ID']),y_axis1,is_smooth=True,linestyle_opts=opts.LineStyleOpts(width=2),
                       label_opts = opts.LabelOpts(is_show=False))
        line.set_global_opts(yaxis_opts=opts.AxisOpts(name='Fluctuation index', name_location='middle',name_gap=30,min_=-1, max_=11, interval = 1,
                                                     name_textstyle_opts=opts.TextStyleOpts(font_family= 'Times New Roman',font_size=16,color='black',),),
                             legend_opts=opts.LegendOpts(pos_left=30, padding = 90, pos_top=-30, orient='horizontal'))
line.render_notebook()  

In [10]:
cluster_num = 3
id_num = -1
line=Line()
line.add_xaxis(x_axis)
for i in y_pred.labels_:
    id_num += 1
    if i == cluster_num:
        y_axis1 = fluc[fluc.columns[3:-2]].iloc[id_num]
        line.add_yaxis(str(fluc.iloc[id_num]['Grid_ID']),y_axis1,is_smooth=True,linestyle_opts=opts.LineStyleOpts(width=2),
                       label_opts = opts.LabelOpts(is_show=False))
        line.set_global_opts(yaxis_opts=opts.AxisOpts(name='Fluctuation index', name_location='middle',name_gap=30,min_=-1, max_=11, interval = 1,
                                                     name_textstyle_opts=opts.TextStyleOpts(font_family= 'Times New Roman',font_size=16,color='black',),),
                             legend_opts=opts.LegendOpts(pos_left=30, padding = 90, pos_top=-30, orient='horizontal'))
line.render_notebook()  

In [11]:
cluster_num = 4
id_num = -1
line=Line()
line.add_xaxis(x_axis)
for i in y_pred.labels_:
    id_num += 1
    if i == cluster_num:
        y_axis1 = fluc[fluc.columns[3:-2]].iloc[id_num]
        line.add_yaxis(str(fluc.iloc[id_num]['Grid_ID']),y_axis1,is_smooth=True,linestyle_opts=opts.LineStyleOpts(width=2),
                       label_opts = opts.LabelOpts(is_show=False))
        line.set_global_opts(yaxis_opts=opts.AxisOpts(name='Fluctuation index', name_location='middle',name_gap=30,min_=-1, max_=11, interval = 1,
                                                     name_textstyle_opts=opts.TextStyleOpts(font_family= 'Times New Roman',font_size=16,color='black',),),
                             legend_opts=opts.LegendOpts(pos_left=30, padding = 90, pos_top=-30, orient='horizontal'))
line.render_notebook()  

In [12]:
cluster_num = 5
id_num = -1
line=Line()
line.add_xaxis(x_axis)
for i in y_pred.labels_:
    id_num += 1
    if i == cluster_num:
        y_axis1 = fluc[fluc.columns[3:-2]].iloc[id_num]
        line.add_yaxis(str(fluc.iloc[id_num]['Grid_ID']),y_axis1,is_smooth=True,linestyle_opts=opts.LineStyleOpts(width=2),
                       label_opts = opts.LabelOpts(is_show=False))
        line.set_global_opts(yaxis_opts=opts.AxisOpts(name='Fluctuation index', name_location='middle',name_gap=30,min_=-1, max_=11, interval = 1,
                                                     name_textstyle_opts=opts.TextStyleOpts(font_family= 'Times New Roman',font_size=16,color='black',),),
                             legend_opts=opts.LegendOpts(pos_left=30, padding = 90, pos_top=-30, orient='horizontal'))
line.render_notebook()  

In [13]:
cluster_num = 6
id_num = -1
line=Line()
line.add_xaxis(x_axis)
for i in y_pred.labels_:
    id_num += 1
    if i == cluster_num:
        y_axis1 = fluc[fluc.columns[3:-2]].iloc[id_num]
        line.add_yaxis(str(fluc.iloc[id_num]['Grid_ID']),y_axis1,is_smooth=True,linestyle_opts=opts.LineStyleOpts(width=2),
                       label_opts = opts.LabelOpts(is_show=False))
        line.set_global_opts(yaxis_opts=opts.AxisOpts(name='Fluctuation index', name_location='middle',name_gap=30,min_=-1, max_=11, interval = 1,
                                                     name_textstyle_opts=opts.TextStyleOpts(font_family= 'Times New Roman',font_size=16,color='black',),),
                             legend_opts=opts.LegendOpts(pos_left=30, padding = 90, pos_top=-30, orient='horizontal'))
line.render_notebook()  

In [14]:
cluster_num = 7
id_num = -1
line=Line()
line.add_xaxis(x_axis)
for i in y_pred.labels_:
    id_num += 1
    if i == cluster_num:
        y_axis1 = fluc[fluc.columns[3:-2]].iloc[id_num]
        line.add_yaxis(str(fluc.iloc[id_num]['Grid_ID']),y_axis1,is_smooth=True,linestyle_opts=opts.LineStyleOpts(width=2),
                       label_opts = opts.LabelOpts(is_show=False))
        line.set_global_opts(yaxis_opts=opts.AxisOpts(name='Fluctuation index', name_location='middle',name_gap=30,min_=-1, max_=11, interval = 1,
                                                     name_textstyle_opts=opts.TextStyleOpts(font_family= 'Times New Roman',font_size=16,color='black',),),
                             legend_opts=opts.LegendOpts(pos_left=30, padding = 90, pos_top=-30, orient='horizontal'))
line.render_notebook()  

In [15]:
cluster_num = 8
id_num = -1
line=Line()
line.add_xaxis(x_axis)
for i in y_pred.labels_:
    id_num += 1
    if i == cluster_num:
        y_axis1 = fluc[fluc.columns[3:-2]].iloc[id_num]
        line.add_yaxis(str(fluc.iloc[id_num]['Grid_ID']),y_axis1,is_smooth=True,linestyle_opts=opts.LineStyleOpts(width=2),
                       label_opts = opts.LabelOpts(is_show=False))
        line.set_global_opts(yaxis_opts=opts.AxisOpts(name='Fluctuation index', name_location='middle',name_gap=30,min_=-1, max_=11, interval = 1,
                                                     name_textstyle_opts=opts.TextStyleOpts(font_family= 'Times New Roman',font_size=16,color='black',),),
                             legend_opts=opts.LegendOpts(pos_left=30, padding = 90, pos_top=-30, orient='horizontal'))
line.render_notebook()  

In [16]:
cluster_num = 9
id_num = -1
line=Line()
line.add_xaxis(x_axis)
for i in y_pred.labels_:
    id_num += 1
    if i == cluster_num:
        y_axis1 = fluc[fluc.columns[3:-2]].iloc[id_num]
        line.add_yaxis(str(fluc.iloc[id_num]['Grid_ID']),y_axis1,is_smooth=True,linestyle_opts=opts.LineStyleOpts(width=2),
                       label_opts = opts.LabelOpts(is_show=False))
        line.set_global_opts(yaxis_opts=opts.AxisOpts(name='Fluctuation index', name_location='middle',name_gap=30,min_=-1, max_=11, interval = 1,
                                                     name_textstyle_opts=opts.TextStyleOpts(font_family= 'Times New Roman',font_size=16,color='black',),),
                             legend_opts=opts.LegendOpts(pos_left=30, padding = 90, pos_top=-30, orient='horizontal'))
line.render_notebook()  

In [17]:
cluster_num = 10
id_num = -1
line=Line()
line.add_xaxis(x_axis)
for i in y_pred.labels_:
    id_num += 1
    if i == cluster_num:
        y_axis1 = fluc[fluc.columns[3:-2]].iloc[id_num]
        line.add_yaxis(str(fluc.iloc[id_num]['Grid_ID']),y_axis1,is_smooth=True,linestyle_opts=opts.LineStyleOpts(width=2),
                       label_opts = opts.LabelOpts(is_show=False))
        line.set_global_opts(yaxis_opts=opts.AxisOpts(name='Fluctuation index', name_location='middle',name_gap=30,min_=-1, max_=11, interval = 1,
                                                     name_textstyle_opts=opts.TextStyleOpts(font_family= 'Times New Roman',font_size=16,color='black',),),
                             legend_opts=opts.LegendOpts(pos_left=30, padding = 90, pos_top=-30, orient='horizontal'))
line.render_notebook()  

In [18]:
cluster_num = 11
id_num = -1
line=Line()
line.add_xaxis(x_axis)
for i in y_pred.labels_:
    id_num += 1
    if i == cluster_num:
        y_axis1 = fluc[fluc.columns[3:-2]].iloc[id_num]
        line.add_yaxis(str(fluc.iloc[id_num]['Grid_ID']),y_axis1,is_smooth=True,linestyle_opts=opts.LineStyleOpts(width=2),
                       label_opts = opts.LabelOpts(is_show=False))
        line.set_global_opts(yaxis_opts=opts.AxisOpts(name='Fluctuation index', name_location='middle',name_gap=30,min_=-1, max_=11, interval = 1,
                                                     name_textstyle_opts=opts.TextStyleOpts(font_family= 'Times New Roman',font_size=16,color='black',),),
                             legend_opts=opts.LegendOpts(pos_left=30, padding = 90, pos_top=-30, orient='horizontal'))
line.render_notebook()  